In [1]:
import os
import glob
import pickle
import random
import math
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from skimage.io import imread, imsave
from scipy.misc import imresize

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.models import load_model, Model
from keras.optimizers import SGD,RMSprop,adam

import pickle
import pandas
import numpy as np

import keras
from keras.preprocessing import image
from keras import applications
from keras.models import Sequential
from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv3D,Input, ZeroPadding3D, Reshape,LSTM
from keras.layers.convolutional import Convolution2D, Convolution3D, MaxPooling2D, ZeroPadding2D,ZeroPadding3D 
from keras.layers.core import Reshape
import os
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import SGD
from keras.layers import Merge 
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback
from keras import regularizers
from keras import backend as K

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# X = np.load("train_X_normalized_lstm_regression.pkl.npy")
# Y = np.load("train_Y_normalized_lstm_regression.pkl.npy")
X = np.load("train_X_normalized_lstm_delta.pkl.npy")
Y = np.load("train_Y_normalized_lstm_delta.pkl.npy")

In [3]:
X = X.reshape(-1, 60,5,1)

In [4]:
X.shape

(622641, 60, 5, 1)

In [11]:
def simple_cnn_model(X, Y):
    model = Sequential()
    model.add(Convolution2D(filters = 64, kernel_size = (30, 3), init='he_normal', padding = 'same', activation='relu', input_shape=(60, 5, 1)))
    #model.add(MaxPooling2D(pool_size = (1,2), strides = (1,2)))
    model.add(Convolution2D(filters = 128, kernel_size = (10, 2), init='he_normal', padding = 'same', activation='relu'))
    #model.add(MaxPooling2D(pool_size = (1,5), strides = (1,5)))    
    model.add(Convolution2D(filters = 256, kernel_size = (5, 2), init='he_normal', padding = 'same', activation='relu'))
    #model.add(MaxPooling2D(pool_size = (1,2), strides = (1,2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.4))
    model.add(Dense(8, activation = 'softmax'))
    
    adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer= 'sgd', metrics=['accuracy'])
    #earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')
    #datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
    #            shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
    #datagen.fit(x_train)
    #model.fit_generator(datagen.flow(x_train, y_label, batch_size=32),steps_per_epoch = len(x_train)/32,
    #                    epochs=10,validation_data=(x_valid, y_valid))
    
    return model
   


In [12]:
model = simple_cnn_model(X, Y)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 60, 5, 64)         5824      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 60, 5, 128)        163968    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 60, 5, 256)        327936    
_________________________________________________________________
flatten_3 (Flatten)          (None, 76800)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               19661056  
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 2056      
Total para

/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=64, kernel_size=(30, 3), padding="same", activation="relu", input_shape=(60, 5, 1), kernel_initializer="he_normal")`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=128, kernel_size=(10, 2), padding="same", activation="relu", kernel_initializer="he_normal")`
  """
/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=256, kernel_size=(5, 2), padding="same", activation="relu", kernel_initializer="he_normal")`
  import sys


In [ ]:
res = model.fit(X, Y ,batch_size=512, epochs=10,verbose=1, shuffle=True,validation_split = 0.1)

Train on 560376 samples, validate on 62265 samples
Epoch 1/10
560376/560376 [==============================] - 430s 767us/step - loss: 1.9087 - acc: 0.3168 - val_loss: 1.9539 - val_acc: 0.2935
Epoch 2/10
560376/560376 [==============================] - 427s 762us/step - loss: 1.8829 - acc: 0.3205 - val_loss: 1.9743 - val_acc: 0.2961
Epoch 3/10
560376/560376 [==============================] - 427s 762us/step - loss: 1.8783 - acc: 0.3219 - val_loss: 1.9421 - val_acc: 0.2967
Epoch 4/10
560376/560376 [==============================] - 427s 762us/step - loss: 1.8758 - acc: 0.3225 - val_loss: 1.9463 - val_acc: 0.2975
Epoch 5/10
  6656/560376 [..............................] - ETA: 6:47 - loss: 1.8697 - acc: 0.3214

In [ ]:
model.save("CNN_1_delta.h5")